In [ ]:
#!pip install emoji plotly wordcloud nltk

In [ ]:
import nltk
nltk.download("stopwords")

In [ ]:
import plotly.express as px
import os
import pandas as pd
import re
import datetime as time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import emoji
import re

from collections import Counter
from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import stopwords

In [ ]:
_CHAT = "_chat.txt"

whatsapp_df = pd.read_fwf(_CHAT, header=None)
whatsapp_df

In [ ]:
whatsapp_df.info()

In [ ]:
user = []
date = []
message = []

for line in enumerate(open(_CHAT, "r")):
    #print(line)
    try:
        d = re.search("\[(.*?)\]\s", line[1]).group(1)
        date.append(d)
    except:
        date.append("1/1/90 01:01:01") # dummy date
    
    try:
        u = re.search("\]\s(.*?):", line[1]).group(1)
        user.append(u)
    except:
        user.append("")
        
    try:
        message.append(line[1].split(": ", 1)[1].split("\n", 1)[0])#.encode("ascii", "ignore").decode())
    except:
        message.append("")
        
print(line)

In [ ]:
df = pd.DataFrame(zip(date, user, message), columns=["date", "user", "message"])
df["date"] = pd.to_datetime(df.date, format="%d/%m/%y %H:%M:%S")

# Remove events not associated with a sender
df = df[df.user != ""].reset_index(drop=True)
df

In [ ]:
delete_audio = df[df["message"].apply(lambda x: x.encode("ascii", "ignore").decode()) == "audio omitido"]
delete_audio.tail()

In [ ]:
df.drop(delete_audio.index, inplace=True)
df.tail()

##### Which users have the most Chat/messages in the group?

In [ ]:
# Creating a new dataframe by copying the old dataframe
whatsapp_df1 = df.copy()
whatsapp_df1["Number_of_messages"] = [1]*whatsapp_df1.shape[0]
whatsapp_df1.drop(columns="date", inplace=True)

# We are groupby the user and messages together then we will use count() to count the messages for each of user
whatsapp_df1 = whatsapp_df1.groupby("user")["Number_of_messages"].count().sort_values(ascending=False).reset_index() 
whatsapp_df1

In [ ]:
# Using seaborn for Styles 
sns.set_style("darkgrid")

# Resize the  figure size
plt.figure(figsize=(12, 9))

# Here we are ploting the line chart using plt.plot 
plt.plot(whatsapp_df1.user, whatsapp_df1.Number_of_messages, "o--c")  

# In here we are writing the Labels and Title for the plot chart
plt.xlabel("Users")
plt.ylabel("Total number of messages")

plt.title("The highest number of messages send by the user")
plt.legend(["Messages send"]);

In [ ]:
# Formating
sns.set_style("darkgrid")

# The background of the chart
matplotlib.rcParams["font.size"] = 12
matplotlib.rcParams["figure.figsize"] = (12, 9)
matplotlib.rcParams["figure.facecolor"] = "#00000000"
fig, ax = plt.subplots()

# Creating a bar chart
sns.barplot(whatsapp_df1.user,
            whatsapp_df1.Number_of_messages,
            hue="user",
            data=whatsapp_df1,
            dodge=False,
            palette="CMRmap")

# The title of our charts
plt.title("The highest number of messages")

# Change the width of the bar chart plot
def change_width(ax, new_value):
    for patch in ax.patches:
        current_width = patch.get_width()
        diff = current_width - new_value
        # we change the bar width
        patch.set_width(new_value)
        # we recenter the bar
        patch.set_x(patch.get_x() + diff * .5)

change_width(ax, .35)
plt.show()

##### Which emojis use the most by which users?

In [ ]:
# Copy a dataset
whatsapp_df2 = df.copy()

# Count the number of emoji
emoji_ctr = Counter()
emojis_list = map(lambda x: "".join(x.split()), emoji.UNICODE_EMOJI.keys()) #UNICODE_EMOJI class have a thee emoji code inside
r = re.compile("|".join(re.escape(p) for p in emojis_list))
for idx, row in whatsapp_df2.iterrows():
    emojis_found = r.findall(row["message"]) #The findall() is a functions for regex that help to find a matches 
    for emoji_found in emojis_found:
        emoji_ctr[emoji_found] += 1

In [ ]:
# This will help to create or rendered the emoji 
emojis_df = pd.DataFrame() #The reason to use pd.dataframe is we want to put the emojis_df into the dataframe tables
emojis_df["emoji"] = [""] * 10
emojis_df["number_of_Emoji"] = [0] * 10

i = 0
for item in emoji_ctr.most_common(10):
    emojis_df.emoji[i] = item[0]
    emojis_df.number_of_Emoji[i] = int(item[1])
    i += 1

emojis_df

In [ ]:
# This pei chart give us and ideas the overall view of which emoji use the most
fig = px.pie(emojis_df, values="number_of_Emoji", names="emoji", title="Emoji percentage used in chat group")
fig.update_traces(textposition="inside", textinfo="percent+label")
fig.show()

In [ ]:
# Now we want to know which emoji is use the most by each of the users. But since the first results only create 
# emoji and number_emoji in the dataframe now we need to create a dataframe contain user and emoji they use

emoji_ctr = {}#Counter()
for user in whatsapp_df2["user"].unique():
    emoji_ctr[user] = Counter()
    
emojis_list = map(lambda x: "".join(x.split()), emoji.UNICODE_EMOJI.keys())
r = re.compile("|".join(re.escape(p) for p in emojis_list))
for idx, row in whatsapp_df2.iterrows():
    emojis_found = r.findall(row["message"])
    for emoji_found in emojis_found:
        emoji_ctr[row["user"]][emoji_found] += 1
        
#emoji_ctr

In [ ]:
user_emoji_df = pd.DataFrame(emoji_ctr)

for user in emoji_ctr.keys():
    ordered_user_df = user_emoji_df.copy()
    ordered_user_df = ordered_user_df.sort_values(by=user, ascending=False).head(10)
    fig = px.pie(ordered_user_df, names=ordered_user_df.index, values=user, title="Emoji percentage for " + user)
    fig.update_traces(textposition="inside", textinfo="percent+label")
    fig.show()

In [ ]:
## nooooo
emojis_df = whatsapp_df2.copy()
n = whatsapp_df2.message.count()

emojis_df["emoji"] = [""] * n
emojis_df["number_of_Emoji"] = [0] * n

i = 0
for user in whatsapp_df2["user"].unique():
    for item in emoji_ctr[user].most_common(n):
        emojis_df.emoji[i] = item[0]
        emojis_df.number_of_Emoji[i] = int(item[1])
        i += 1

emojis_df

In [ ]:
### noooo
l = emojis_df.user.unique()
for i in range(len(l)):
    dummy_df = emojis_df[emojis_df["user"] == l[i]]
    emojis_list = list([a for b in dummy_df.emoji for a in b])
    emoji_dict = dict(Counter(emojis_list))
    emoji_dict = sorted(emoji_dict.items(), key=lambda x: x[1], reverse=True)
    print("Emoji Distribution for", l[i])
    user_emoji_df = pd.DataFrame(emoji_dict, columns=["emoji", "count"])
    fig = px.pie(user_emoji_df, values="count", names="emoji")
    fig.update_traces(textposition="inside", textinfo="percent+label")
    fig.show()

##### Most active hour in whatsapps

In [ ]:
# Copy a dataframe
whatsapp_df3 = df.copy()

whatsapp_df3["number_of_message"] = [1] * whatsapp_df3.shape[0]
whatsapp_df3["hours"] = whatsapp_df3["date"].apply(lambda x: x.hour)

time_df = whatsapp_df3.groupby("hours").count().reset_index().sort_values(by = "hours")
time_df[["hours", "number_of_message"]]

In [ ]:
#Create the formatting of the graph 
matplotlib.rcParams["font.size"] = 20
matplotlib.rcParams["figure.figsize"] = (20, 8)

# Using the seaborn style 
sns.set_style("darkgrid")
plt.title("Most active hour in whatsapps");
sns.barplot(time_df.hours, time_df.number_of_message,data=time_df, dodge=False)

##### Which month have the highest messages and also the busiest month?

In [ ]:
whatsapp_df4 = df.copy()
whatsapp_df4["Number_of_messages"] = [1] * whatsapp_df4.shape[0]

whatsapp_df4["month"] = whatsapp_df4["date"].apply(lambda x: x.month)  

df_month = whatsapp_df4.groupby("month")["Number_of_messages"].count().sort_values(ascending=False).reset_index()
df_month

In [ ]:
#Formating
sns.set_style("darkgrid")

#The background of the chart
matplotlib.rcParams["font.size"] = 12
matplotlib.rcParams["figure.figsize"] = (12, 9)
matplotlib.rcParams["figure.facecolor"] = "#00000000"
fig, ax = plt.subplots()

#Creating a bar chart
sns.barplot(x=df_month.month,
            y=df_month.Number_of_messages,
            hue="month",
            data=df_month,
            dodge=False,
            palette="pastel")

plt.title("Month that have the highest messages and the busiest month?")

##### Determine which word or text did the user use the most?

In [ ]:
whatsapp_df5 = df.copy()
#Each of the word in the message will be review
word = " ".join(review for review in whatsapp_df5.message)

stop_words = set(stopwords.words("spanish"))

#delete the word/text that are commonly used(eg.the,yes,no,bye,or and is)
#stopwords.update(["the","is","yea","ok","okay","or","bye","no","will","yeah","I","almost","if","me","you","done","want","Ya"])

#Creating a word cloud 
wordcloud = WordCloud(width=500,
                      height =500,
                      stopwords=stop_words,
                      background_color="black",
                      min_font_size=10).generate(word)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
wordcloud.to_image()